In [44]:
#sentiment analysis
import pandas as pd

reddit = pd.read_csv("RedditNews.csv", engine="python", sep=",", error_bad_lines=False)

# Show DataFrame
reddit.head()

,Date,Title
0,7/1/16,IMF chief backs Athens as permanent Olympic host
1,7/1/16,"The president of France says if Brexit won, so..."
2,7/1/16,British Man Who Must Give Police 24 Hours' Not...
3,7/1/16,100+ Nobel laureates urge Greenpeace to stop o...
4,7/1/16,Brazil: Huge spike in number of police killing...


In [45]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
#nltk.download('wordnet')

In [46]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


reddit['cleanText']=reddit['Title'].map(lambda s:preprocess(s))

In [47]:
reddit.head()

,Date,Title,cleanText
0,7/1/16,IMF chief backs Athens as permanent Olympic host,imf chief backs athens permanent olympic host
1,7/1/16,"The president of France says if Brexit won, so...",president france says brexit donald trump
2,7/1/16,British Man Who Must Give Police 24 Hours' Not...,british man must give police hours notice sex ...
3,7/1/16,100+ Nobel laureates urge Greenpeace to stop o...,nobel laureates urge greenpeace stop opposing ...
4,7/1/16,Brazil: Huge spike in number of police killing...,brazil huge spike number police killings rio a...


In [48]:
reddit['Date'] =  pd.to_datetime(reddit['Date'])
reddit.head()

,Date,Title,cleanText
0,2016-07-01,IMF chief backs Athens as permanent Olympic host,imf chief backs athens permanent olympic host
1,2016-07-01,"The president of France says if Brexit won, so...",president france says brexit donald trump
2,2016-07-01,British Man Who Must Give Police 24 Hours' Not...,british man must give police hours notice sex ...
3,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...,nobel laureates urge greenpeace stop opposing ...
4,2016-07-01,Brazil: Huge spike in number of police killing...,brazil huge spike number police killings rio a...


In [49]:
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
#pip install vaderSentiment 

In [51]:
#pip install --upgrade vaderSentimen


In [52]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [53]:
preprocess = []
analyzer = SentimentIntensityAnalyzer()
for clean in reddit['cleanText']:
    vs = analyzer.polarity_scores(clean)
    preprocess.append(vs)


In [54]:
#positive sentiment: compound score >= 0.05
#neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#negative sentiment: compound score <= -0.05

In [55]:
df3 = pd.DataFrame(preprocess)

In [56]:
df4 = pd.concat([reddit,df3], axis=1)

In [57]:
djia = pd.read_csv("DJIA.csv", engine="python", sep=",", error_bad_lines=False)

In [58]:
djia.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [59]:
djia['Date'] =  pd.to_datetime(djia['Date'], format='%Y-%m-%d')

In [60]:
merge = pd.merge(df4,djia, how='inner', on = 'Date')

In [61]:
merge.head()

,Date,Title,cleanText,compound,neg,neu,pos,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,IMF chief backs Athens as permanent Olympic host,imf chief backs athens permanent olympic host,-0.0516,0.167,0.833,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-07-01,"The president of France says if Brexit won, so...",president france says brexit donald trump,0.0000,0.000,1.000,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
2,2016-07-01,British Man Who Must Give Police 24 Hours' Not...,british man must give police hours notice sex ...,-0.8210,0.413,0.587,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
3,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...,nobel laureates urge greenpeace stop opposing ...,-0.2960,0.268,0.732,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
4,2016-07-01,Brazil: Huge spike in number of police killing...,brazil huge spike number police killings rio a...,-0.4404,0.319,0.426,0.255,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141


In [62]:
returns = (merge.Close - merge.Close.shift(1))/merge.Close.shift(1)
returns.head(51)

0          NaN
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.000000
21    0.000000
22    0.000000
23    0.000000
24   -0.001080
25    0.000000
26    0.000000
27    0.000000
28    0.000000
29    0.000000
30    0.000000
31    0.000000
32    0.000000
33    0.000000
34    0.000000
35    0.000000
36    0.000000
37    0.000000
38    0.000000
39    0.000000
40    0.000000
41    0.000000
42    0.000000
43    0.000000
44    0.000000
45    0.000000
46    0.000000
47    0.000000
48    0.000000
49   -0.013124
50    0.000000
Name: Close, dtype: float64

In [63]:
merge['returns'] = returns

In [64]:
merge.head()

,Date,Title,cleanText,compound,neg,neu,pos,Open,High,Low,Close,Volume,Adj Close,returns
0,2016-07-01,IMF chief backs Athens as permanent Olympic host,imf chief backs athens permanent olympic host,-0.0516,0.167,0.833,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,NaN
1,2016-07-01,"The president of France says if Brexit won, so...",president france says brexit donald trump,0.0000,0.000,1.000,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,0.0
2,2016-07-01,British Man Who Must Give Police 24 Hours' Not...,british man must give police hours notice sex ...,-0.8210,0.413,0.587,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,0.0
3,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...,nobel laureates urge greenpeace stop opposing ...,-0.2960,0.268,0.732,0.000,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,0.0
4,2016-07-01,Brazil: Huge spike in number of police killing...,brazil huge spike number police killings rio a...,-0.4404,0.319,0.426,0.255,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,0.0


In [67]:
df5 = pd.concat(df4['Date'], axis=0, join='outer')

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [71]:
pd.concat([df4,djia], axis=1)

,Date,Title,cleanText,compound,neg,neu,pos,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,IMF chief backs Athens as permanent Olympic host,imf chief backs athens permanent olympic host,-0.0516,0.167,0.833,0.000,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000.0,17949.369141
1,2016-07-01,"The president of France says if Brexit won, so...",president france says brexit donald trump,0.0000,0.000,1.000,0.000,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000.0,17929.990234
2,2016-07-01,British Man Who Must Give Police 24 Hours' Not...,british man must give police hours notice sex ...,-0.8210,0.413,0.587,0.000,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000.0,17694.679688
3,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...,nobel laureates urge greenpeace stop opposing ...,-0.2960,0.268,0.732,0.000,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000.0,17409.720703
4,2016-07-01,Brazil: Huge spike in number of police killing...,brazil huge spike number police killings rio a...,-0.4404,0.319,0.426,0.255,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000.0,17140.240234
5,2016-07-01,Austria's highest court annuls presidential el...,austria highest court annuls presidential elec...,-0.3182,0.187,0.813,0.000,2016-06-24,17946.630859,17946.630859,17356.339844,17400.750000,239000000.0,17400.750000
6,2016-07-01,"Facebook wins privacy case, can track any Belg...",facebook wins privacy case track belgian wants...,0.5859,0.000,0.676,0.324,2016-06-23,17844.109375,18011.070312,17844.109375,18011.070312,98070000.0,18011.070312
7,2016-07-01,Switzerland denies Muslim girls citizenship af...,switzerland denies muslim girls citizenship re...,-0.6124,0.208,0.792,0.000,2016-06-22,17832.669922,17920.160156,17770.359375,17780.830078,89440000.0,17780.830078
8,2016-07-01,China kills millions of innocent meditators fo...,china kills millions innocent meditators organ...,-0.2732,0.294,0.504,0.202,2016-06-21,17827.330078,17877.839844,17799.800781,17829.730469,85130000.0,17829.730469
9,2016-07-01,France Cracks Down on Factory Farms - A viral ...,france cracks factory farms viral video campai...,-0.8402,0.299,0.632,0.069,2016-06-20,17736.869141,17946.359375,17736.869141,17804.869141,99380000.0,17804.869141
